Example: Distributed deep learning training using TensorFlow and Keras with HorovodRunner
=========================================================================================

from
https://docs.databricks.com/applications/machine-learning/train-model/distributed-training/mnist-tensorflow-keras.html

Set up correct versions of packages for our needs

In [ ]:
pip install tensorflow==1.15 
pip show tensorflow

  

>     Found existing installation: tensorflow 2.3.0
>     Uninstalling tensorflow-2.3.0:
>       Would remove:
>         /databricks/conda/envs/databricks-ml-gpu/bin/estimator_ckpt_converter
>         /databricks/conda/envs/databricks-ml-gpu/bin/saved_model_cli
>         /databricks/conda/envs/databricks-ml-gpu/bin/tensorboard
>         /databricks/conda/envs/databricks-ml-gpu/bin/tf_upgrade_v2
>         /databricks/conda/envs/databricks-ml-gpu/bin/tflite_convert
>         /databricks/conda/envs/databricks-ml-gpu/bin/toco
>         /databricks/conda/envs/databricks-ml-gpu/bin/toco_from_protos
>         /databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages/tensorflow-2.3.0.dist-info/*
>         /databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages/tensorflow/*
>     ERROR: Exception:
>     Traceback (most recent call last):
>       File "/databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages/pip/_internal/cli/base_command.py", line 186, in _main
>         status = self.run(options, args)
>       File "/databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages/pip/_internal/commands/uninstall.py", line 79, in run
>         auto_confirm=options.yes, verbose=self.verbosity > 0,
>       File "/databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages/pip/_internal/req/req_install.py", line 687, in uninstall
>         uninstalled_pathset.remove(auto_confirm, verbose)
>       File "/databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages/pip/_internal/req/req_uninstall.py", line 388, in remove
>         if auto_confirm or self._allowed_to_proceed(verbose):
>       File "/databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages/pip/_internal/req/req_uninstall.py", line 431, in _allowed_to_proceed
>         return ask('Proceed (y/n)? ', ('y', 'n')) == 'y'
>       File "/databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages/pip/_internal/utils/misc.py", line 240, in ask
>         _check_no_input(message)
>       File "/databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages/pip/_internal/utils/misc.py", line 232, in _check_no_input
>         message
>     Exception: No input was expected ($PIP_NO_INPUT set); question: Proceed (y/n)? 
>     Collecting tensorflow==1.15
>       Downloading tensorflow-1.15.0-cp37-cp37m-manylinux2010_x86_64.whl (412.3 MB)
>     Collecting tensorboard<1.16.0,>=1.15.0
>       Downloading tensorboard-1.15.0-py3-none-any.whl (3.8 MB)
>     Requirement already satisfied: termcolor>=1.1.0 in /databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages (from tensorflow==1.15) (1.1.0)
>     Requirement already satisfied: wrapt>=1.11.1 in /databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages (from tensorflow==1.15) (1.11.2)
>     Requirement already satisfied: keras-preprocessing>=1.0.5 in /databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages (from tensorflow==1.15) (1.1.2)
>     Requirement already satisfied: astor>=0.6.0 in /databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages (from tensorflow==1.15) (0.8.0)
>     Requirement already satisfied: numpy<2.0,>=1.16.0 in /databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages (from tensorflow==1.15) (1.18.1)
>     Requirement already satisfied: opt-einsum>=2.3.2 in /databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages (from tensorflow==1.15) (3.3.0)
>     Collecting gast==0.2.2
>       Downloading gast-0.2.2.tar.gz (10 kB)
>     Requirement already satisfied: six>=1.10.0 in /databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages (from tensorflow==1.15) (1.14.0)
>     Requirement already satisfied: protobuf>=3.6.1 in /databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages (from tensorflow==1.15) (3.11.4)
>     Requirement already satisfied: grpcio>=1.8.6 in /databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages (from tensorflow==1.15) (1.27.2)
>     Collecting tensorflow-estimator==1.15.1
>       Downloading tensorflow_estimator-1.15.1-py2.py3-none-any.whl (503 kB)
>     Collecting keras-applications>=1.0.8
>       Downloading Keras_Applications-1.0.8-py3-none-any.whl (50 kB)
>     Requirement already satisfied: google-pasta>=0.1.6 in /databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages (from tensorflow==1.15) (0.2.0)
>     Requirement already satisfied: absl-py>=0.7.0 in /databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages (from tensorflow==1.15) (0.9.0)
>     Requirement already satisfied: wheel>=0.26 in /databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages (from tensorflow==1.15) (0.34.2)
>     Requirement already satisfied: markdown>=2.6.8 in /databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages (from tensorboard<1.16.0,>=1.15.0->tensorflow==1.15) (3.1.1)
>     Requirement already satisfied: setuptools>=41.0.0 in /databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages (from tensorboard<1.16.0,>=1.15.0->tensorflow==1.15) (45.2.0.post20200210)
>     Requirement already satisfied: werkzeug>=0.11.15 in /databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages (from tensorboard<1.16.0,>=1.15.0->tensorflow==1.15) (1.0.0)
>     Requirement already satisfied: h5py in /databricks/conda/envs/databricks-ml-gpu/lib/python3.7/site-packages (from keras-applications>=1.0.8->tensorflow==1.15) (2.10.0)
>     Building wheels for collected packages: gast
>       Building wheel for gast (setup.py): started
>       Building wheel for gast (setup.py): finished with status 'done'
>       Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7539 sha256=dff4f82b6a0364dcfb9ae8f35ade6178dc888a22198e108d3192db9b3b586df8
>       Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
>     Successfully built gast
>     ERROR: spark-tensorflow-distributor 0.1.0 has requirement tensorflow>=2.1.0, but you'll have tensorflow 1.15.0 which is incompatible.
>     Installing collected packages: tensorboard, gast, tensorflow-estimator, keras-applications, tensorflow
>       Attempting uninstall: tensorboard
>         Found existing installation: tensorboard 2.3.0
>         Uninstalling tensorboard-2.3.0:
>           Successfully uninstalled tensorboard-2.3.0
>       Attempting uninstall: gast
>         Found existing installation: gast 0.3.3
>         Uninstalling gast-0.3.3:
>           Successfully uninstalled gast-0.3.3
>       Attempting uninstall: tensorflow-estimator
>         Found existing installation: tensorflow-estimator 2.3.0
>         Uninstalling tensorflow-estimator-2.3.0:
>           Successfully uninstalled tensorflow-estimator-2.3.0
>       Attempting uninstall: tensorflow
>         Found existing installation: tensorflow 2.3.0
>         Uninstalling tensorflow-2.3.0:
>           Successfully uninstalled tensorflow-2.3.0
>     Successfully installed gast-0.2.2 keras-applications-1.0.8 tensorboard-1.15.0 tensorflow-1.15.0 tensorflow-estimator-1.15.1

  

Set up checkpoint location

In [ ]:
import os
import time
 
checkpoint_dir = '/dbfs/ml/MNISTDemo/train/{}/'.format(time.time())
 
os.makedirs(checkpoint_dir)

  

  

Create function to prepare data

In [ ]:
def get_dataset(num_classes, rank=0, size=1):
  from tensorflow import keras
  
  (x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data('MNIST-data-%d' % rank)
  x_train = x_train[rank::size]
  y_train = y_train[rank::size]
  x_test = x_test[rank::size]
  y_test = y_test[rank::size]
  x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
  x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
  x_train = x_train.astype('float32')
  x_test = x_test.astype('float32')
  x_train /= 255
  x_test /= 255
  y_train = keras.utils.to_categorical(y_train, num_classes)
  y_test = keras.utils.to_categorical(y_test, num_classes)
  return (x_train, y_train), (x_test, y_test)

  

  

Create function to train model

In [ ]:
def get_model(num_classes):
  from tensorflow.keras import models
  from tensorflow.keras import layers
  
  model = models.Sequential()
  model.add(layers.Conv2D(32, kernel_size=(3, 3),
                   activation='relu',
                   input_shape=(28, 28, 1)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Dropout(0.25))
  model.add(layers.Flatten())
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(num_classes, activation='softmax'))
  return model

  

  

Run training on single node

In [ ]:
# Specify training parameters
batch_size = 128
epochs = 20
num_classes = 10        
 
 
def train(learning_rate=1.0):
  from tensorflow import keras
  
  (x_train, y_train), (x_test, y_test) = get_dataset(num_classes)
  print(x_train.shape)
  print(y_train.shape)
  print(x_test.shape)
  print(y_test.shape)
  model = get_model(num_classes)
 
  # Specify the optimizer (Adadelta in this example), using the learning rate input parameter of the function so that Horovod can adjust the learning rate during training
  optimizer = keras.optimizers.Adam(lr=learning_rate)
 
  model.compile(optimizer=optimizer,
                loss='categorical_crossentropy',
                metrics=['accuracy'])
 
  model.fit(x_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            verbose=2,
            validation_data=(x_test, y_test))


In [ ]:
  # run the training
  train(learning_rate=0.1)

  

>     (60000, 28, 28, 1)
>     (60000, 10)
>     (10000, 28, 28, 1)
>     (10000, 10)
>     Train on 60000 samples, validate on 10000 samples
>     Epoch 1/20
>     60000/60000 - 51s - loss: 3.8806 - acc: 0.1048 - val_loss: 2.3070 - val_acc: 0.0958
>     Epoch 2/20

  

Migrate to HorovodRunner for distributed training

In [ ]:
def train_hvd(learning_rate=1.0):
  # Import tensorflow modules to each worker
  from tensorflow.keras import backend as K
  from tensorflow.keras.models import Sequential
  import tensorflow as tf
  from tensorflow import keras
  import horovod.tensorflow.keras as hvd
  
  # Initialize Horovod
  hvd.init()
 
  # Pin GPU to be used to process local rank (one GPU per process)
  # These steps are skipped on a CPU cluster
  gpus = tf.config.experimental.list_physical_devices('GPU')
  for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
  if gpus:
    tf.config.experimental.set_visible_devices(gpus[hvd.local_rank()], 'GPU')
 
  # Call the get_dataset function you created, this time with the Horovod rank and size
  print("HVD RANK AND SIZE")
  print(hvd.rank())
  print(hvd.size())
  
  (x_train, y_train), (x_test, y_test) = get_dataset(num_classes, hvd.rank(), hvd.size())
  print(x_train.shape)
  print(y_train.shape)
  print(x_test.shape)
  print(y_test.shape)
  model = get_model(num_classes)
 
  # Adjust learning rate based on number of GPUs
  optimizer = keras.optimizers.Adam(lr=learning_rate * hvd.size())
  # Use the Horovod Distributed Optimizer
  optimizer = hvd.DistributedOptimizer(optimizer)
 
  model.compile(optimizer=optimizer,
                loss='categorical_crossentropy',
                metrics=['accuracy'])
 
  # Create a callback to broadcast the initial variable states from rank 0 to all other processes.
  # This is required to ensure consistent initialization of all workers when training is started with random weights or restored from a checkpoint.
  callbacks = [hvd.callbacks.BroadcastGlobalVariablesCallback(0)]
 
  # Save checkpoints only on worker 0 to prevent conflicts between workers
  if hvd.rank() == 0:
      callbacks.append(keras.callbacks.ModelCheckpoint(checkpoint_dir + '/checkpoint-{epoch}.ckpt', save_weights_only = True))
 
  model.fit(x_train, y_train,
            batch_size=batch_size,
            callbacks=callbacks,
            epochs=epochs,
            verbose=2)

In [ ]:
from sparkdl import HorovodRunner
 
hr = HorovodRunner(np=2)
hr.run(train_hvd, learning_rate=0.1)

  

>     HorovodRunner will stream all training logs to notebook cell output. If there are too many logs, you
>     can adjust the log level in your train method. Or you can set driver_log_verbosity to
>     'log_callback_only' and use a HorovodRunner log  callback on the first worker to get concise
>     progress updates.
>     The global names read or written to by the pickled function are {'checkpoint_dir', 'num_classes', 'batch_size', 'epochs', 'get_model', 'print', 'get_dataset'}.
>     The pickled object size is 3648 bytes.
>
>     ### How to enable Horovod Timeline? ###
>     HorovodRunner has the ability to record the timeline of its activity with Horovod  Timeline. To
>     record a Horovod Timeline, set the `HOROVOD_TIMELINE` environment variable  to the location of the
>     timeline file to be created. You can then open the timeline file  using the chrome://tracing
>     facility of the Chrome browser.
>
>     Start training.
>     [1,1]<stderr>:Traceback (most recent call last):
>     [1,1]<stderr>:  File "<string>", line 1, in <module>
>     [1,1]<stderr>:  File "/databricks/.python_edge_libs/sparkdl/horovod/runner.py", line 222, in wrapped_main
>     [1,1]<stderr>:    return_value = main(**kwargs)
>     [1,1]<stderr>:  File "<command-1173833896357973>", line 7, in train_hvd
>     [1,1]<stderr>:  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages/horovod/tensorflow/__init__.py", line 28, in <module>
>     [1,1]<stderr>:    from horovod.tensorflow.mpi_ops import allgather, broadcast, _allreduce
>     [1,1]<stderr>:  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages/horovod/tensorflow/mpi_ops.py", line 49, in <module>
>     [1,1]<stderr>:    MPI_LIB = _load_library('mpi_lib' + get_ext_suffix())
>     [1,1]<stderr>:  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages/horovod/tensorflow/mpi_ops.py", line 45, in _load_library
>     [1,1]<stderr>:    library = load_library.load_op_library(filename)
>     [1,1]<stderr>:  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages/tensorflow_core/python/framework/load_library.py", line 61, in load_op_library
>     [1,1]<stderr>:    lib_handle = py_tf.TF_LoadLibrary(library_filename)
>     [1,1]<stderr>:tensorflow.python.framework.errors_impl.NotFoundError: libtensorflow_framework.so.2: cannot open shared object file: No such file or directory
>     --------------------------------------------------------------------------
>     Primary job  terminated normally, but 1 process returned
>     a non-zero exit code. Per user-direction, the job has been aborted.
>     --------------------------------------------------------------------------
>     [1,0]<stderr>:Traceback (most recent call last):
>     [1,0]<stderr>:  File "<string>", line 1, in <module>
>     [1,0]<stderr>:  File "/databricks/.python_edge_libs/sparkdl/horovod/runner.py", line 222, in wrapped_main
>     [1,0]<stderr>:    return_value = main(**kwargs)
>     [1,0]<stderr>:  File "<command-1173833896357973>", line 7, in train_hvd
>     [1,0]<stderr>:  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages/horovod/tensorflow/__init__.py", line 28, in <module>
>     [1,0]<stderr>:    from horovod.tensorflow.mpi_ops import allgather, broadcast, _allreduce
>     [1,0]<stderr>:  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages/horovod/tensorflow/mpi_ops.py", line 49, in <module>
>     [1,0]<stderr>:    MPI_LIB = _load_library('mpi_lib' + get_ext_suffix())
>     [1,0]<stderr>:  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages/horovod/tensorflow/mpi_ops.py", line 45, in _load_library
>     [1,0]<stderr>:    library = load_library.load_op_library(filename)
>     [1,0]<stderr>:  File "/local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages/tensorflow_core/python/framework/load_library.py", line 61, in load_op_library
>     [1,0]<stderr>:    lib_handle = py_tf.TF_LoadLibrary(library_filename)
>     [1,0]<stderr>:tensorflow.python.framework.errors_impl.NotFoundError: libtensorflow_framework.so.2: cannot open shared object file: No such file or directory
>     --------------------------------------------------------------------------
>     mpirun detected that one or more processes exited with non-zero status, thus causing
>     the job to be terminated. The first process to do so was:
>
>       Process name: [[24816,1],1]
>       Exit code:    1
>     --------------------------------------------------------------------------

In [ ]:
install tensorflow-cpu==1.15.*
%pip install tensorflow-gpu==1.15.*


  

>     Python interpreter will be restarted.
>     Requirement already satisfied: tensorflow-cpu==1.15.* in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (1.15.0)
>     Requirement already satisfied: tensorboard<1.16.0,>=1.15.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-cpu==1.15.*) (1.15.0)
>     Requirement already satisfied: termcolor>=1.1.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-cpu==1.15.*) (1.1.0)
>     Requirement already satisfied: wrapt>=1.11.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-cpu==1.15.*) (1.11.2)
>     Requirement already satisfied: keras-preprocessing>=1.0.5 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-cpu==1.15.*) (1.1.2)
>     Requirement already satisfied: astor>=0.6.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-cpu==1.15.*) (0.8.0)
>     Requirement already satisfied: numpy<2.0,>=1.16.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-cpu==1.15.*) (1.18.1)
>     Requirement already satisfied: opt-einsum>=2.3.2 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-cpu==1.15.*) (3.3.0)
>     Requirement already satisfied: gast==0.2.2 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-cpu==1.15.*) (0.2.2)
>     Requirement already satisfied: six>=1.10.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-cpu==1.15.*) (1.14.0)
>     Requirement already satisfied: protobuf>=3.6.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-cpu==1.15.*) (3.11.4)
>     Requirement already satisfied: grpcio>=1.8.6 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-cpu==1.15.*) (1.27.2)
>     Requirement already satisfied: tensorflow-estimator==1.15.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-cpu==1.15.*) (1.15.1)
>     Requirement already satisfied: keras-applications>=1.0.8 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-cpu==1.15.*) (1.0.8)
>     Requirement already satisfied: google-pasta>=0.1.6 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-cpu==1.15.*) (0.2.0)
>     Requirement already satisfied: absl-py>=0.7.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-cpu==1.15.*) (0.9.0)
>     Requirement already satisfied: wheel>=0.26 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-cpu==1.15.*) (0.34.2)
>     Requirement already satisfied: markdown>=2.6.8 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorboard<1.16.0,>=1.15.0->tensorflow-cpu==1.15.*) (3.1.1)
>     Requirement already satisfied: setuptools>=41.0.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorboard<1.16.0,>=1.15.0->tensorflow-cpu==1.15.*) (45.2.0.post20200210)
>     Requirement already satisfied: werkzeug>=0.11.15 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorboard<1.16.0,>=1.15.0->tensorflow-cpu==1.15.*) (1.0.0)
>     Requirement already satisfied: h5py in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from keras-applications>=1.0.8->tensorflow-cpu==1.15.*) (2.10.0)
>     Python interpreter will be restarted.
>     Python interpreter will be restarted.
>     Collecting tensorflow-gpu==1.15.*
>       Using cached tensorflow_gpu-1.15.4-cp37-cp37m-manylinux2010_x86_64.whl (411.0 MB)
>     Requirement already satisfied: tensorboard<1.16.0,>=1.15.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-gpu==1.15.*) (1.15.0)
>     Requirement already satisfied: termcolor>=1.1.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-gpu==1.15.*) (1.1.0)
>     Requirement already satisfied: wrapt>=1.11.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-gpu==1.15.*) (1.11.2)
>     Requirement already satisfied: wheel>=0.26; python_version >= "3" in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-gpu==1.15.*) (0.34.2)
>     Requirement already satisfied: numpy<1.19.0,>=1.16.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-gpu==1.15.*) (1.18.1)
>     Requirement already satisfied: keras-preprocessing>=1.0.5 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-gpu==1.15.*) (1.1.2)
>     Requirement already satisfied: astor>=0.6.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-gpu==1.15.*) (0.8.0)
>     Requirement already satisfied: opt-einsum>=2.3.2 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-gpu==1.15.*) (3.3.0)
>     Requirement already satisfied: six>=1.10.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-gpu==1.15.*) (1.14.0)
>     Requirement already satisfied: gast==0.2.2 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-gpu==1.15.*) (0.2.2)
>     Requirement already satisfied: protobuf>=3.6.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-gpu==1.15.*) (3.11.4)
>     Requirement already satisfied: grpcio>=1.8.6 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-gpu==1.15.*) (1.27.2)
>     Requirement already satisfied: tensorflow-estimator==1.15.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-gpu==1.15.*) (1.15.1)
>     Requirement already satisfied: keras-applications>=1.0.8 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-gpu==1.15.*) (1.0.8)
>     Requirement already satisfied: google-pasta>=0.1.6 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-gpu==1.15.*) (0.2.0)
>     Requirement already satisfied: absl-py>=0.7.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow-gpu==1.15.*) (0.9.0)
>     Requirement already satisfied: markdown>=2.6.8 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorboard<1.16.0,>=1.15.0->tensorflow-gpu==1.15.*) (3.1.1)
>     Requirement already satisfied: setuptools>=41.0.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorboard<1.16.0,>=1.15.0->tensorflow-gpu==1.15.*) (45.2.0.post20200210)
>     Requirement already satisfied: werkzeug>=0.11.15 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorboard<1.16.0,>=1.15.0->tensorflow-gpu==1.15.*) (1.0.0)
>     Requirement already satisfied: h5py in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from keras-applications>=1.0.8->tensorflow-gpu==1.15.*) (2.10.0)
>     Installing collected packages: tensorflow-gpu
>     Successfully installed tensorflow-gpu-1.15.4
>     Python interpreter will be restarted.

In [ ]:
install https://databricks-prod-cloudfront.cloud.databricks.com/artifacts/tensorflow/runtime-7.x/tensorflow-1.15.3-cp37-cp37m-linux_x86_64.whl

  

>     Python interpreter will be restarted.
>     Collecting tensorflow==1.15.3
>       Using cached https://databricks-prod-cloudfront.cloud.databricks.com/artifacts/tensorflow/runtime-7.x/tensorflow-1.15.3-cp37-cp37m-linux_x86_64.whl (310.3 MB)
>     Requirement already satisfied: tensorboard<1.16.0,>=1.15.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow==1.15.3) (1.15.0)
>     Requirement already satisfied: termcolor>=1.1.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow==1.15.3) (1.1.0)
>     Requirement already satisfied: wrapt>=1.11.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow==1.15.3) (1.11.2)
>     Requirement already satisfied: wheel>=0.26; python_version >= "3" in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow==1.15.3) (0.34.2)
>     Requirement already satisfied: keras-preprocessing>=1.0.5 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow==1.15.3) (1.1.2)
>     Requirement already satisfied: astor>=0.6.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow==1.15.3) (0.8.0)
>     Requirement already satisfied: numpy<2.0,>=1.16.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow==1.15.3) (1.18.1)
>     Requirement already satisfied: opt-einsum>=2.3.2 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow==1.15.3) (3.3.0)
>     Requirement already satisfied: gast==0.2.2 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow==1.15.3) (0.2.2)
>     Requirement already satisfied: six>=1.10.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow==1.15.3) (1.14.0)
>     Requirement already satisfied: protobuf>=3.6.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow==1.15.3) (3.11.4)
>     Requirement already satisfied: grpcio>=1.8.6 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow==1.15.3) (1.27.2)
>     Requirement already satisfied: tensorflow-estimator==1.15.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow==1.15.3) (1.15.1)
>     Requirement already satisfied: keras-applications>=1.0.8 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow==1.15.3) (1.0.8)
>     Requirement already satisfied: google-pasta>=0.1.6 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow==1.15.3) (0.2.0)
>     Requirement already satisfied: absl-py>=0.7.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorflow==1.15.3) (0.9.0)
>     Requirement already satisfied: markdown>=2.6.8 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorboard<1.16.0,>=1.15.0->tensorflow==1.15.3) (3.1.1)
>     Requirement already satisfied: setuptools>=41.0.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorboard<1.16.0,>=1.15.0->tensorflow==1.15.3) (45.2.0.post20200210)
>     Requirement already satisfied: werkzeug>=0.11.15 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from tensorboard<1.16.0,>=1.15.0->tensorflow==1.15.3) (1.0.0)
>     Requirement already satisfied: h5py in /local_disk0/.ephemeral_nfs/envs/pythonEnv-aa69c28f-c2d6-4acc-b96b-82dc9a083c73/lib/python3.7/site-packages (from keras-applications>=1.0.8->tensorflow==1.15.3) (2.10.0)
>     ERROR: spark-tensorflow-distributor 0.1.0 has requirement tensorflow>=2.1.0, but you'll have tensorflow 1.15.3 which is incompatible.
>     Installing collected packages: tensorflow
>       Attempting uninstall: tensorflow
>         Found existing installation: tensorflow 2.3.0
>         Uninstalling tensorflow-2.3.0:
>           Successfully uninstalled tensorflow-2.3.0
>     Successfully installed tensorflow-1.15.3
>     Python interpreter will be restarted.

In [ ]:
import tensorflow as tf
print(tf.__version__)

  

>     1.15.3